# Customer Support ticket Dataset

Here we will load a dataset containing customer support Tickets. There are ca. 8500 customer tickets, and each tickets as values such as customer details, purchased product, description, ticket types and various ticket labels. #todo link


## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the AI-Lab](../main_config.ipynb).

## Setup

### Open Secure Configuration Storage

In [1]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

Output()

Box(children=(Box(children=(Label(value='Configuration Store', layout=Layout(border_bottom='solid 1px', border…

## Download data

We will access the dataset on [Kaggle](https://www.kaggle.com/) using [kagglehub](https://github.com/Kaggle/kagglehub).

In [2]:
!pip install kagglehub
import kagglehub

#todo switch to S3 download, put filtered data there first
# Download latest version
download_path = kagglehub.dataset_download("suraj520/customer-support-ticket-dataset")

print("Path to dataset files:", download_path)


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Path to dataset files: /home/jupyter/.cache/kagglehub/datasets/suraj520/customer-support-ticket-dataset/versions/1


Then we will load the data into a csv and convert it into a pandas dataframe for ease of use.

In [3]:
from pathlib import Path
import pandas as pd

csv_file = next(Path(download_path).glob("*"))
df = pd.read_csv(csv_file)

Now we are ready to upload the data to the Exasol Database:

In [7]:
from exasol.nb_connector.connections import open_pyexasol_connection

with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    conn.execute(f"""
    CREATE OR REPLACE TABLE "{ai_lab_config.db_schema}"."CUSTOMER_SUPPORT_TICKETS" (
        "TICKET_ID" INTEGER,
        "CUSTOMER_NAME" VARCHAR(2000000),
        "CUSTOMER_EMAIL" VARCHAR(2000000),
        "CUSTOMER_AGE" INTEGER,
        "CUSTOMER_GENDER" VARCHAR(2000000),
        "PRODUCT_PURCHASED" VARCHAR(2000000),
        "DATE_OF_PURCHASE" VARCHAR(2000000),
        "TICKET_TYPE" VARCHAR(2000000),
        "TICKET_SUBJECT" VARCHAR(2000000),
        "TICKET_DESCRIPTION" VARCHAR(2000000),
        "TICKET_STATUS" VARCHAR(2000000),
        "RESOLUTION" VARCHAR(2000000),
        "TICKET_PRIORITY" VARCHAR(2000000),
        "TICKET_CHANNEL" VARCHAR(2000000),
        "FIRST_RESPONSE_TIME" VARCHAR(2000000),
        "TIME_TO_RESOLUTION" VARCHAR(2000000),
        "CUSTOMER_SATISFACTION_RATING" VARCHAR(2000000)
    );
    """)
    conn.import_from_pandas(df, table=(ai_lab_config.db_schema,"CUSTOMER_SUPPORT_TICKETS"))
